# Data Preprocessing for Air Quality Prediction

This notebook demonstrates the data preprocessing pipeline for the air quality prediction project.

## Objectives
1. Load raw air quality data
2. Handle missing values and outliers
3. Filter data for target cities
4. Apply data cleaning techniques
5. Prepare data for feature engineering

## Preprocessing Steps
- Remove rows with null AQI values
- Treat outliers using IQR and Z-score methods
- Apply SMOTE for data balancing
- Scale numerical features
- Save processed data for next steps


In [3]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    """Install package using pip if not already installed."""
    try:
        __import__(package)
        print(f"{package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
required_packages = [
    "pandas",
    "numpy", 
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "imbalanced-learn",
    "tqdm",
    "joblib"
]

for package in required_packages:
    install_package(package)

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from scipy import stats
import warnings
import os

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("Preprocessing libraries imported successfully!")


pandas is already installed
numpy is already installed
matplotlib is already installed
seaborn is already installed
Installing scikit-learn...
Installing imbalanced-learn...
tqdm is already installed
joblib is already installed
Preprocessing libraries imported successfully!


## 1. Data Loading


In [4]:
# Load raw data
data_path = "../data/raw/"

# Check available files and load the dataset
print("🔍 Checking available dataset files...")
if os.path.exists(data_path):
    files = os.listdir(data_path)
    print(f"Available files: {files}")
    
    # Load the main dataset - city_day (daily city-level data)
    try:
        # Try loading city_day file (main dataset for our project)
        if 'city_day' in files:
            # Check if it's CSV or Excel format
            if 'city_day.csv' in files:
                df_raw = pd.read_csv(data_path + 'city_day.csv')
                print("✅ Loaded city_day.csv successfully!")
            else:
                # Try as Excel file
                df_raw = pd.read_excel(data_path + 'city_day')
                print("✅ Loaded city_day Excel file successfully!")
            
            print(f"📊 Raw dataset shape: {df_raw.shape}")
            print(f"📋 Columns: {list(df_raw.columns)}")
            
        else:
            print("⚠️ city_day file not found. Using first available file...")
            available_files = [f for f in files if f.endswith(('.csv', '.xlsx', '.xls')) or '.' not in f]
            if available_files:
                first_file = available_files[0]
                if first_file.endswith('.csv'):
                    df_raw = pd.read_csv(data_path + first_file)
                else:
                    df_raw = pd.read_excel(data_path + first_file)
                print(f"✅ Loaded {first_file} as main dataset")
            else:
                print("❌ No suitable data files found")
                df_raw = pd.DataFrame()
                
    except Exception as e:
        print(f"❌ Error loading dataset: {e}")
        df_raw = pd.DataFrame()
        
else:
    print(f"❌ Data directory not found: {data_path}")
    df_raw = pd.DataFrame()

# Import preprocessing class
import sys
sys.path.append('../src')
from data_preprocessing import AirQualityPreprocessor

# Initialize preprocessor with actual data path
if not df_raw.empty:
    # Save a copy for the preprocessor to use
    df_raw.to_csv(data_path + "temp_raw_data.csv", index=False)
    preprocessor = AirQualityPreprocessor(data_path + "temp_raw_data.csv")
    print("✅ Preprocessor initialized with actual dataset!")
else:
    print("⚠️ Preprocessor initialized with placeholder path")
    preprocessor = AirQualityPreprocessor(data_path + "air_quality_data.csv")


🔍 Checking available dataset files...
Available files: ['city_day.csv', 'city_hour.csv', 'stations.csv', 'station_day.csv', 'station_hour.csv']
⚠️ city_day file not found. Using first available file...
✅ Loaded city_day.csv as main dataset


INFO:data_preprocessing:Initialized AirQualityPreprocessor with data path: ../data/raw/temp_raw_data.csv


✅ Preprocessor initialized with actual dataset!


## 2. Data Exploration and Quality Assessment


In [5]:
# Explore the loaded dataset
if not df_raw.empty:
    print("📊 RAW DATASET ANALYSIS")
    print("=" * 60)
    
    # Basic information
    print(f"Dataset Shape: {df_raw.shape}")
    print(f"Memory Usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Display first few rows
    print("\n📋 First 5 rows:")
    print(df_raw.head())
    
    # Data types and missing values
    print("\n📝 Data Types and Missing Values:")
    missing_info = pd.DataFrame({
        'Data Type': df_raw.dtypes,
        'Missing Values': df_raw.isnull().sum(),
        'Missing %': (df_raw.isnull().sum() / len(df_raw) * 100).round(2)
    })
    print(missing_info)
    
    # Check for target cities
    target_cities = ['Delhi', 'Bangalore', 'Kolkata', 'Hyderabad', 'Chennai', 'Visakhapatnam']
    
    if 'City' in df_raw.columns:
        available_cities = df_raw['City'].unique()
        print(f"\n🏙️ Total Cities in Dataset: {len(available_cities)}")
        print(f"First 10 cities: {available_cities[:10]}")
        
        # Check target cities
        target_cities_found = [city for city in target_cities if city in available_cities]
        print(f"\n🎯 Target Cities Found ({len(target_cities_found)}/{len(target_cities)}):")
        for city in target_cities:
            if city in available_cities:
                count = len(df_raw[df_raw['City'] == city])
                print(f"  ✅ {city}: {count:,} records")
            else:
                print(f"  ❌ {city}: Not found")
        
        # Check for similar city names
        if len(target_cities_found) < len(target_cities):
            print(f"\n🔍 Checking for similar city names...")
            for target in target_cities:
                if target not in available_cities:
                    similar = [city for city in available_cities if target.lower() in city.lower() or city.lower() in target.lower()]
                    if similar:
                        print(f"  💡 {target} → Similar: {similar}")
    else:
        print(f"\n⚠️ 'City' column not found. Available columns: {list(df_raw.columns)}")
    
    # Check AQI column
    if 'AQI' in df_raw.columns:
        print(f"\n🌬️ AQI Analysis:")
        print(f"  Range: {df_raw['AQI'].min():.2f} to {df_raw['AQI'].max():.2f}")
        print(f"  Mean: {df_raw['AQI'].mean():.2f}")
        print(f"  Median: {df_raw['AQI'].median():.2f}")
        print(f"  Missing AQI values: {df_raw['AQI'].isnull().sum():,} ({df_raw['AQI'].isnull().sum()/len(df_raw)*100:.2f}%)")
        
        # AQI distribution
        print(f"\n📊 AQI Distribution:")
        aqi_stats = df_raw['AQI'].describe()
        print(aqi_stats)
        
    else:
        print(f"\n⚠️ 'AQI' column not found. Available columns: {list(df_raw.columns)}")
    
    # Check date column
    if 'Date' in df_raw.columns:
        df_raw['Date'] = pd.to_datetime(df_raw['Date'])
        print(f"\n📅 Date Range:")
        print(f"  From: {df_raw['Date'].min()}")
        print(f"  To: {df_raw['Date'].max()}")
        print(f"  Duration: {(df_raw['Date'].max() - df_raw['Date'].min()).days} days")
    else:
        print(f"\n⚠️ 'Date' column not found. Available columns: {list(df_raw.columns)}")

else:
    print("❌ No data loaded. Please check file paths and formats.")


📊 RAW DATASET ANALYSIS
Dataset Shape: (29531, 16)
Memory Usage: 7.71 MB

📋 First 5 rows:
        City        Date  PM2.5  PM10     NO    NO2    NOx  NH3     CO    SO2  \
0  Ahmedabad  2015-01-01    NaN   NaN   0.92  18.22  17.15  NaN   0.92  27.64   
1  Ahmedabad  2015-01-02    NaN   NaN   0.97  15.69  16.46  NaN   0.97  24.55   
2  Ahmedabad  2015-01-03    NaN   NaN  17.40  19.30  29.70  NaN  17.40  29.07   
3  Ahmedabad  2015-01-04    NaN   NaN   1.70  18.48  17.97  NaN   1.70  18.59   
4  Ahmedabad  2015-01-05    NaN   NaN  22.10  21.42  37.76  NaN  22.10  39.33   

       O3  Benzene  Toluene  Xylene  AQI AQI_Bucket  
0  133.36     0.00     0.02    0.00  NaN        NaN  
1   34.06     3.68     5.50    3.77  NaN        NaN  
2   30.70     6.80    16.40    2.25  NaN        NaN  
3   36.08     4.43    10.14    1.00  NaN        NaN  
4   39.31     7.01    18.89    2.78  NaN        NaN  

📝 Data Types and Missing Values:
           Data Type  Missing Values  Missing %
City          obje

## 3. Data Preprocessing Pipeline


In [6]:
# Step 1: Filter data for target cities
print("🏙️ FILTERING FOR TARGET CITIES")
print("=" * 50)

target_cities = ['Delhi', 'Bangalore', 'Kolkata', 'Hyderabad', 'Chennai', 'Visakhapatnam']

if not df_raw.empty and 'City' in df_raw.columns:
    # Check available cities and find matches
    available_cities = df_raw['City'].unique()
    print(f"Available cities: {len(available_cities)}")
    
    # Find exact matches and similar names
    city_mapping = {}
    for target in target_cities:
        if target in available_cities:
            city_mapping[target] = target
        else:
            # Look for similar names (case-insensitive)
            similar = [city for city in available_cities 
                      if target.lower() in city.lower() or city.lower() in target.lower()]
            if similar:
                city_mapping[target] = similar[0]  # Take first match
                print(f"  💡 Mapping: {target} → {similar[0]}")
    
    # Filter data for target cities
    target_city_data = []
    for target, actual_city in city_mapping.items():
        city_data = df_raw[df_raw['City'] == actual_city].copy()
        city_data['City'] = target  # Standardize city name
        target_city_data.append(city_data)
        print(f"  ✅ {target}: {len(city_data):,} records")
    
    if target_city_data:
        df_filtered = pd.concat(target_city_data, ignore_index=True)
        print(f"\n📊 Filtered dataset shape: {df_filtered.shape}")
        print(f"   Original: {df_raw.shape}")
        print(f"   Filtered: {df_filtered.shape}")
    else:
        print("❌ No target cities found in dataset")
        df_filtered = df_raw.copy()
        
else:
    print("⚠️ Cannot filter cities - City column not found")
    df_filtered = df_raw.copy()

print(f"\n🎯 Target cities data summary:")
if 'City' in df_filtered.columns:
    city_counts = df_filtered['City'].value_counts()
    print(city_counts)


🏙️ FILTERING FOR TARGET CITIES
Available cities: 26
  ✅ Delhi: 2,009 records
  ✅ Kolkata: 814 records
  ✅ Hyderabad: 2,006 records
  ✅ Chennai: 2,009 records
  ✅ Visakhapatnam: 1,462 records

📊 Filtered dataset shape: (8300, 16)
   Original: (29531, 16)
   Filtered: (8300, 16)

🎯 Target cities data summary:
City
Delhi            2009
Chennai          2009
Hyderabad        2006
Visakhapatnam    1462
Kolkata           814
Name: count, dtype: int64


In [7]:
# Step 2: Handle missing values - Remove rows with null AQI
print("\n🧹 HANDLING MISSING VALUES")
print("=" * 50)

if not df_filtered.empty and 'AQI' in df_filtered.columns:
    initial_shape = df_filtered.shape
    print(f"Initial dataset shape: {initial_shape}")
    
    # Remove rows with null AQI (target variable)
    df_no_null_aqi = df_filtered.dropna(subset=['AQI'])
    null_aqi_removed = initial_shape[0] - df_no_null_aqi.shape[0]
    
    print(f"✅ Removed {null_aqi_removed:,} rows with null AQI")
    print(f"   Remaining rows: {df_no_null_aqi.shape[0]:,}")
    
    # Check other missing values
    print(f"\n📊 Missing values after AQI cleaning:")
    missing_after = df_no_null_aqi.isnull().sum()
    missing_after = missing_after[missing_after > 0]
    
    if len(missing_after) > 0:
        print("Columns with missing values:")
        for col, count in missing_after.items():
            percentage = (count / len(df_no_null_aqi)) * 100
            print(f"  {col}: {count:,} ({percentage:.2f}%)")
    else:
        print("✅ No missing values remaining!")
    
    df_cleaned = df_no_null_aqi.copy()
    
else:
    print("⚠️ Cannot clean missing values - AQI column not found")
    df_cleaned = df_filtered.copy()

print(f"\n📊 Final cleaned dataset shape: {df_cleaned.shape}")



🧹 HANDLING MISSING VALUES
Initial dataset shape: (8300, 16)
✅ Removed 612 rows with null AQI
   Remaining rows: 7,688

📊 Missing values after AQI cleaning:
Columns with missing values:
  PM2.5: 20 (0.26%)
  PM10: 1,909 (24.83%)
  NO: 18 (0.23%)
  NO2: 25 (0.33%)
  NOx: 15 (0.20%)
  NH3: 697 (9.07%)
  CO: 13 (0.17%)
  SO2: 124 (1.61%)
  O3: 103 (1.34%)
  Benzene: 329 (4.28%)
  Toluene: 303 (3.94%)
  Xylene: 3,140 (40.84%)

📊 Final cleaned dataset shape: (7688, 16)


In [8]:
# Step 3: Outlier Detection and Treatment
print("\n🔍 OUTLIER DETECTION AND TREATMENT")
print("=" * 50)

if not df_cleaned.empty:
    # Select numeric columns for outlier analysis
    numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove Date columns if present
    numeric_cols = [col for col in numeric_cols if 'Date' not in col and col not in ['year', 'month', 'day']]
    
    print(f"Analyzing outliers in {len(numeric_cols)} numeric columns:")
    print(f"Columns: {numeric_cols}")
    
    # Detect outliers using IQR method
    outlier_summary = {}
    
    for col in numeric_cols:
        if col in df_cleaned.columns:
            Q1 = df_cleaned[col].quantile(0.25)
            Q3 = df_cleaned[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = df_cleaned[(df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)]
            outlier_count = len(outliers)
            outlier_percentage = (outlier_count / len(df_cleaned)) * 100
            
            outlier_summary[col] = {
                'count': outlier_count,
                'percentage': outlier_percentage,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound
            }
            
            print(f"  {col}: {outlier_count:,} outliers ({outlier_percentage:.2f}%)")
    
    # Show outlier treatment options
    print(f"\n📊 Outlier Treatment Summary:")
    for col, info in outlier_summary.items():
        if info['count'] > 0:
            print(f"  {col}: {info['count']} outliers (bounds: {info['lower_bound']:.2f} to {info['upper_bound']:.2f})")
    
    # Apply outlier treatment (capping method)
    df_treated = df_cleaned.copy()
    outliers_treated = 0
    
    for col, info in outlier_summary.items():
        if info['count'] > 0:
            # Cap outliers instead of removing them
            df_treated[col] = np.where(df_treated[col] < info['lower_bound'], info['lower_bound'], df_treated[col])
            df_treated[col] = np.where(df_treated[col] > info['upper_bound'], info['upper_bound'], df_treated[col])
            outliers_treated += info['count']
    
    if outliers_treated > 0:
        print(f"\n✅ Treated {outliers_treated} outlier values using capping method")
    else:
        print(f"\n✅ No outliers found in the dataset")
    
    df_final = df_treated.copy()
    
else:
    print("⚠️ No data available for outlier analysis")
    df_final = df_cleaned.copy()

print(f"\n📊 Final processed dataset shape: {df_final.shape}")



🔍 OUTLIER DETECTION AND TREATMENT
Analyzing outliers in 13 numeric columns:
Columns: ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI']
  PM2.5: 680 outliers (8.84%)
  PM10: 345 outliers (4.49%)
  NO: 877 outliers (11.41%)
  NO2: 201 outliers (2.61%)
  NOx: 695 outliers (9.04%)
  NH3: 424 outliers (5.52%)
  CO: 509 outliers (6.62%)
  SO2: 329 outliers (4.28%)
  O3: 335 outliers (4.36%)
  Benzene: 418 outliers (5.44%)
  Toluene: 553 outliers (7.19%)
  Xylene: 248 outliers (3.23%)
  AQI: 569 outliers (7.40%)

📊 Outlier Treatment Summary:
  PM2.5: 680 outliers (bounds: -35.98 to 145.92)
  PM10: 345 outliers (bounds: -96.72 to 352.20)
  NO: 877 outliers (bounds: -14.41 to 39.84)
  NO2: 201 outliers (bounds: -26.37 to 88.47)
  NOx: 695 outliers (bounds: -22.71 to 79.33)
  NH3: 424 outliers (bounds: -30.83 to 84.03)
  CO: 509 outliers (bounds: -0.46 to 2.18)
  SO2: 329 outliers (bounds: -6.64 to 26.53)
  O3: 335 outliers (bounds: -11.43 to

In [9]:
# Step 4: Save processed data
print("\n💾 SAVING PROCESSED DATA")
print("=" * 50)

if not df_final.empty:
    # Create output directory if it doesn't exist
    output_dir = "../data/processed/"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the processed dataset
    output_file = output_dir + "cleaned_data.csv"
    df_final.to_csv(output_file, index=False)
    
    print(f"✅ Processed data saved to: {output_file}")
    print(f"   Shape: {df_final.shape}")
    print(f"   File size: {os.path.getsize(output_file) / 1024**2:.2f} MB")
    
    # Display final summary
    print(f"\n📊 PREPROCESSING SUMMARY")
    print("=" * 50)
    print(f"Original dataset: {df_raw.shape if not df_raw.empty else 'N/A'}")
    print(f"After city filtering: {df_filtered.shape if not df_filtered.empty else 'N/A'}")
    print(f"After missing value removal: {df_cleaned.shape if not df_cleaned.empty else 'N/A'}")
    print(f"Final processed dataset: {df_final.shape}")
    
    # City-wise summary
    if 'City' in df_final.columns:
        print(f"\n🏙️ City-wise record counts:")
        city_counts = df_final['City'].value_counts()
        for city, count in city_counts.items():
            print(f"  {city}: {count:,} records")
    
    # AQI summary
    if 'AQI' in df_final.columns:
        print(f"\n🌬️ Final AQI Statistics:")
        print(f"  Range: {df_final['AQI'].min():.2f} to {df_final['AQI'].max():.2f}")
        print(f"  Mean: {df_final['AQI'].mean():.2f}")
        print(f"  Std: {df_final['AQI'].std():.2f}")
        
else:
    print("❌ No processed data to save")

print(f"\n🎉 Preprocessing completed successfully!")
print(f"Next step: Run feature engineering notebook (03_feature_engineering.ipynb)")



💾 SAVING PROCESSED DATA
✅ Processed data saved to: ../data/processed/cleaned_data.csv
   Shape: (7688, 16)
   File size: 0.76 MB

📊 PREPROCESSING SUMMARY
Original dataset: (29531, 16)
After city filtering: (8300, 16)
After missing value removal: (7688, 16)
Final processed dataset: (7688, 16)

🏙️ City-wise record counts:
  Delhi: 1,999 records
  Chennai: 1,884 records
  Hyderabad: 1,880 records
  Visakhapatnam: 1,171 records
  Kolkata: 754 records

🌬️ Final AQI Statistics:
  Range: 22.00 to 346.50
  Mean: 148.85
  Std: 89.39

🎉 Preprocessing completed successfully!
Next step: Run feature engineering notebook (03_feature_engineering.ipynb)
